In [ ]:
from qualtran.bloqs.factoring.mod_exp import ModExp
from qualtran.drawing import get_musical_score_data, draw_musical_score

modexp_small = ModExp(base=3, mod=16, exp_bitsize=3, x_bitsize=2048)
ms = get_musical_score_data(modexp_small.decompose_bloq())

draw_musical_score(ms)

In [ ]:
from sympy import Symbol
from typing import Any

from qualtran.drawing.musical_score import MusicalScoreEncoder
from qualtran._infra.registers import Side

class BloqEncoder(MusicalScoreEncoder):

  def default(self, o: Any) -> Any:
    if isinstance(o, (Symbol)):
      return f'Symbol({o})'
    if isinstance(o, complex):
      return f'{o.real}+{o.imag}i'
    if isinstance(o, Side):
      return 'Side.LEFT' if o == Side.LEFT else 'Side.RIGHT' if o == Side.RIGHT else 'Side.THRU'

    try:
      return super().default(o)
    except:
      return 'NOT_SERIALIZABLE'

In [ ]:
import attrs
import hashlib
import json

from qualtran import Adjoint, CompositeBloq, Controlled
from qualtran.bloqs.rotations.programmable_rotation_gate_array import ProgrammableRotationGateArray

def bloq_attrs(bloq):
  if isinstance(bloq, CompositeBloq):
    return {}
  if isinstance(bloq, ProgrammableRotationGateArray):
    return {}
  if isinstance(bloq, (Adjoint, Controlled)):
    return bloq_attrs(bloq.subbloq)

  return attrs.asdict(bloq)

def bloq_filename(bloq):
  attrs_dict = bloq_attrs(bloq)
  attrs_keys = list(attrs_dict.keys())
  attrs_keys.sort()

  prefix = ''
  if isinstance(bloq, Adjoint):
    prefix = 'Adjoint_'
  if isinstance(bloq, Controlled):
    prefix = 'Controlled_'

  attrs_list = [
    [key, attrs_dict[key]]
    for key in attrs_keys
  ] if attrs_keys else []
  unhashed = json.dumps(attrs_list, cls=BloqEncoder)

  return prefix + hashlib.md5(unhashed.encode(), usedforsecurity=False).hexdigest() + '.json'

In [ ]:
import json
from pathlib import Path

from qualtran_dev_tools.notebook_specs import NB_BY_SECTION

docs_by_section = {
  section[0]: {
    notebook_spec.title: list(
      {
        'name': bloq_spec.bloq_cls.__name__,
        'examples': list(
          {
            'name': example.name,
            'filename': bloq_filename(example.make())
          }
          for example in bloq_spec.examples
        )
      }
      for bloq_spec in notebook_spec.bloq_specs
    )
    for notebook_spec in section[1]
  }
  for section in NB_BY_SECTION
}

Path('ui_export').mkdir(parents=True, exist_ok=True)

with open('ui_export/navigation.json', 'w') as f:
  json.dump(docs_by_section, f, indent=2)

In [ ]:
import os
from pathlib import Path

from qualtran_dev_tools.notebook_specs import NB_BY_SECTION
from qualtran_dev_tools.parse_docstrings import get_markdown_docstring

for section in NB_BY_SECTION:
  for notebook_spec in section[1]:
    for bloq_spec in notebook_spec.bloq_specs:
      Path(f'ui_export/{bloq_spec.bloq_cls.__name__}').mkdir(parents=True, exist_ok=True)

      doc_name = f'ui_export/{bloq_spec.bloq_cls.__name__}/docs.txt'
      if not os.path.isfile(doc_name):
        with open(doc_name, 'w') as doc_file:
          doc_file.write('\n'.join(get_markdown_docstring(bloq_spec.bloq_cls)))

In [ ]:
import json
import os
from pathlib import Path

from qualtran_dev_tools.all_call_graph import get_all_call_graph
from qualtran_dev_tools.notebook_specs import NB_BY_SECTION
from qualtran import Adjoint, Controlled
from qualtran.drawing.musical_score import get_musical_score_data

examples = [
  example
  for section in NB_BY_SECTION
  for notebook_spec in section[1]
  for bloq_spec in notebook_spec.bloq_specs
  for example in bloq_spec.examples
]

call_graph = get_all_call_graph(examples)

def bloq_score(bloq):
  try:
    return get_musical_score_data(bloq.decompose_bloq())
  except:
    return None

def bloq_name(bloq):
  if (isinstance(bloq, (Adjoint, Controlled))):
    return bloq_name(bloq.subbloq)

  return bloq.__class__.__name__

def write_example(bloq):
  file_name = f'ui_export/{bloq_name(bloq)}/{bloq_filename(bloq)}'
  if not os.path.isfile(file_name):
    bloq_dict = {
      'name': str(bloq),
      'attrs': bloq_attrs(bloq),
      'msd': bloq_score(bloq),
      'callees': list(
        {
          'name': bloq_name(child_bloq),
          'filename': bloq_filename(child_bloq)
        }
        for child_bloq in call_graph.neighbors(bloq)
      )
    }

    Path(f'ui_export/{bloq_name(bloq)}').mkdir(parents=True, exist_ok=True)

    with open(file_name, 'w') as f:
      json.dump(bloq_dict, f, indent=2, cls=BloqEncoder)

for bloq, _ in call_graph.nodes.items():
  write_example(bloq)